In [1]:
import os
import pandas as pd
import janitor
import gzip
import json
from tqdm.notebook import tqdm

# see contabo server
data_path = "../output/cache_payload/stars"

In [2]:
data = []
for date in tqdm(os.listdir(data_path)):
# for date in os.listdir(data_path):
    # print(date)
    for pkg_date_snapshot in os.listdir("/".join([data_path, date])):
    # for pkg_date_snapshot in os.listdir("/".join([data_path, date]))[:5]:
        slug = pkg_date_snapshot.replace(f"{date}_", "").replace(".json.gz", "")
        # try:
            # user, repo = slug.split("_", 1)
            # user, repo = slug.split("_")
        # except ValueError:
        #     print(f"Error with slug: {slug}")
    # for pkg_date_snapshot in os.listdir("/".join([data_path, date])):
        full_path = "/".join([data_path, date, pkg_date_snapshot])
        if full_path.endswith(".json.gz"):
            with gzip.open(full_path, "rb") as f:
                stargazer_data = json.load(f)
            stars = len(stargazer_data)
            # print(stars)
            
            # data.append([slug, date, stars])
            data.append([slug, date, stars])            

  0%|          | 0/33 [00:00<?, ?it/s]

In [3]:
df = (
    pd.DataFrame(data, columns=["slug", "date", "stars"])
    # pd.DataFrame(data, columns=["slug", "date", "user", "repo", "stars"])
    .sort_values(["slug", "date"], ignore_index=True)
)
# df.to_csv("repo_stars_timeseries.csv", index=False)
df

,slug,date,stars
0,0riion_py-sls-lambda-toolkit,2023-05-09,0
1,0riion_py-sls-lambda-toolkit,2023-05-10,0
2,0riion_py-sls-lambda-toolkit,2023-05-11,0
3,0riion_py-sls-lambda-toolkit,2023-05-12,0
4,0riion_py-sls-lambda-toolkit,2023-05-13,25
...,...,...,...
17447,zjupgx_diverge,2023-06-03,0
17448,zjupgx_diverge,2023-06-04,0
17449,zjupgx_diverge,2023-06-05,0
17450,zjupgx_diverge,2023-06-06,0


In [4]:
# # Shows what an empty payload contains:
# _path = "cache_payload/stars/2023-05-28/2023-05-28_7RedViolin_pySigma-backend-sentinelone.json.gz"
# with gzip.open(_path, "rb") as f:
#     json_data = json.load(f)
# json_data

In [5]:
# # Shows what a (non-empty) payload contains:
# _path = "cache_payload/stars/2023-05-28/2023-05-28_0riion_py-sls-lambda-toolkit.json.gz"
# with gzip.open(_path, "rb") as f:
#     json_data = json.load(f)
# json_data

In [6]:
# Read in 100 random sample
df_random_sample = (
    pd.read_csv("../input/random_sample.csv")
    .assign(slug=lambda df: df["github"].str.replace("https://github.com/", ""))
    .assign(slug=lambda df: df["slug"].apply(lambda x: x[:-1] if x.endswith("/") else x))
    .assign(slug=lambda df: df["slug"].str.lower().str.strip())
    .assign(fileslug=lambda df: df["slug"].str.replace("/", "_"))
    .assign(treated=1)
    # Assign the two types
    .assign(boughtstars=lambda df: (df.index < 25).astype(int))
    .assign(treated2=lambda df: df["boughtstars"] + df["treated"])
)
df_random_sample.head(3)

,pkg,return_code,github_url,homepage,earliest_release,gh_url_check,github,pypi,label,slug,fileslug,treated,boughtstars,treated2
0,bird-ospf-link-db-parser,200.0,https://github.com/Andrew-Dickinson/bird-ospf-...,NaN,2023-04-29T07:23:44,1.0,https://github.com/Andrew-Dickinson/bird-ospf-...,https://pypi.org/project/bird-ospf-link-db-par...,bird-ospf-link-db-parser\r\nhttps://github.com...,andrew-dickinson/bird-ospf-link-db-parser,andrew-dickinson_bird-ospf-link-db-parser,1,1,2
1,asciicli,200.0,https://github.com/mrq-andras/asciicli,https://github.com/mrq-andras/asciicli,2023-04-28T07:22:55,1.0,https://github.com/mrq-andras/asciicli,https://pypi.org/project/asciicli/#history,asciicli\r\nhttps://github.com/mrq-andras/asci...,mrq-andras/asciicli,mrq-andras_asciicli,1,1,2
2,bdpotentiometer,200.0,https://github.com/bond-anton/BDPotentiometer,https://github.com/bond-anton/BDPotentiometer,2023-04-27T06:35:18,1.0,https://github.com/bond-anton/BDPotentiometer,https://pypi.org/project/bdpotentiometer/#history,bdpotentiometer\r\nhttps://github.com/bond-ant...,bond-anton/bdpotentiometer,bond-anton_bdpotentiometer,1,1,2


In [7]:
# Get assignment status
df = (
    df
    .rename_column("slug", "fileslug")
    # Normalize fileslug string to lowercase before merge
    .assign(fileslug=lambda df: df["fileslug"].str.lower().str.strip())
    .merge(
        df_random_sample.select_columns("fileslug", "treated", "boughtstars", "treated2"),
        how="left",
        on="fileslug",
        validate="m:1",
        indicator=True,
    )
    .assign(treated=lambda df: df["treated"].fillna(0).apply(int))
    .assign(boughtstars=lambda df: df["boughtstars"].fillna(0).apply(int))
    .assign(treated2=lambda df: df["treated2"].fillna(0).apply(int))
    .assign(date=lambda df: pd.to_datetime(df["date"]))
)
# Assert that 100 repos are assigned as treated
assert 100 == len(df.drop_duplicates("fileslug").query("treated==1"))
assert 25 == len(df.drop_duplicates("fileslug").query("treated2==2"))

df

,fileslug,date,stars,treated,boughtstars,treated2,_merge
0,0riion_py-sls-lambda-toolkit,2023-05-09,0,1,1,2,both
1,0riion_py-sls-lambda-toolkit,2023-05-10,0,1,1,2,both
2,0riion_py-sls-lambda-toolkit,2023-05-11,0,1,1,2,both
3,0riion_py-sls-lambda-toolkit,2023-05-12,0,1,1,2,both
4,0riion_py-sls-lambda-toolkit,2023-05-13,25,1,1,2,both
...,...,...,...,...,...,...,...
17447,zjupgx_diverge,2023-06-03,0,0,0,0,left_only
17448,zjupgx_diverge,2023-06-04,0,0,0,0,left_only
17449,zjupgx_diverge,2023-06-05,0,0,0,0,left_only
17450,zjupgx_diverge,2023-06-06,0,0,0,0,left_only


In [8]:
df.to_csv("../output/repo_stars_timeseries.csv", index=False)